In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
from faker import Faker

In [3]:
client = QdrantClient(host="localhost", port=6333)
client

In [4]:
my_collection = "first_collection"
client.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE)
)

True

In [12]:
data = np.random.uniform(low=-1.0, high=1.0, size=(1_000, 100))
index = list(range(len(data)))

In [ ]:
data[:2, :10]

In [10]:
index[:-10]

SyntaxError: incomplete input (2260647823.py, line 1)

In [14]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist()
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [16]:
client.retrieve(
    collection_name=my_collection,
    ids=[10, 14, 500],
    with_vectors=True
)

[Record(id=10, payload={}, vector=[-0.011372728, 0.119516686, 0.083762184, -0.02461968, 0.029361432, -0.17252031, 0.031493176, 0.16062932, 0.13710298, 0.06220982, -0.08088169, 0.16570649, -0.012590312, -0.12681308, -0.12606704, 0.08646026, 0.041210733, -0.08251057, 0.17204265, -0.08245342, -0.018393032, 0.039341573, 0.10318609, -0.13219635, -0.13893276, 0.07218921, 0.04498553, -0.036600243, 0.11096045, 0.06324486, 0.16248618, -0.062345643, -0.118802615, -0.11066045, -0.10128554, -0.032076705, -0.0080039045, -0.068044975, -0.0069813253, 0.13910457, -0.056120656, 0.094681844, 0.1725404, -0.050081328, -0.12026461, 0.11330385, 0.08327986, -0.10025092, 0.115296066, -0.05580845, 0.0640668, -0.06856974, 0.16219778, 0.07556238, 0.08351952, -0.04834676, 0.0041681495, -0.14655124, 0.04794741, 0.05720926, -0.056891944, 0.0786138, -0.16428155, -0.11897887, -0.09901335, 0.061752193, -0.077501945, 0.14963296, -0.03251117, -0.14589761, 0.12838978, -0.076444164, 0.03643285, 0.09045592, 0.033275757, 0.

In [17]:
fake_data = Faker()
fake_data.name(), fake_data.address()

('Kimberly Rose', '01619 Rivera Harbor\nPort Shane, MI 78702')

In [19]:
payload = []

for i in range(1_000):
    payload.append(
        {
            "artist": fake_data.name(),
            "song": " ".join(fake_data.words()),
            "url_song": fake_data.url(),
            "year": fake_data.year(),
            "country": fake_data.country(),
        }
    )
payload[:3]

[{'artist': 'Joseph Gonzales',
  'song': 'participant improve suggest',
  'url_song': 'http://www.green-campbell.com/',
  'year': '1984',
  'country': 'Albania'},
 {'artist': 'Andrea Alvarez',
  'song': 'book again look',
  'url_song': 'http://hill-carney.biz/',
  'year': '2008',
  'country': 'Oman'},
 {'artist': 'Dominic Washington',
  'song': 'at when student',
  'url_song': 'http://shaw-gutierrez.com/',
  'year': '1988',
  'country': 'Tanzania'}]

In [20]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist(),
        payloads=payload
    )
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [21]:
## Semantic Search

In [22]:
living_la_vida_loca = np.random.uniform(low=-1.0, high=1.0, size=(100)).tolist()

In [23]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    limit=10
)

[ScoredPoint(id=583, version=1, score=0.35136586, payload={'artist': 'Ashley Marshall', 'country': 'Cambodia', 'song': 'less ground stop', 'url_song': 'https://brown.com/', 'year': '1974'}, vector=None, shard_key=None),
 ScoredPoint(id=934, version=1, score=0.3210162, payload={'artist': 'Robert Smith', 'country': 'Cape Verde', 'song': 'seven as start', 'url_song': 'http://johnson-cohen.com/', 'year': '1990'}, vector=None, shard_key=None),
 ScoredPoint(id=747, version=1, score=0.29749677, payload={'artist': 'Samantha Herrera', 'country': 'Saint Lucia', 'song': 'first cultural firm', 'url_song': 'https://flores.com/', 'year': '1999'}, vector=None, shard_key=None),
 ScoredPoint(id=949, version=1, score=0.28410935, payload={'artist': 'Andre Rivera', 'country': 'Czech Republic', 'song': 'shoulder field risk', 'url_song': 'https://schwartz.info/', 'year': '1996'}, vector=None, shard_key=None),
 ScoredPoint(id=283, version=1, score=0.27747548, payload={'artist': 'Denise Farrell', 'country': '

In [28]:
aussie_songs = models.Filter(
    must=[
        models.FieldCondition(
            key="country", match=models.MatchValue(value="Australia")
        )
    ]
)
aussie_songs

Filter(should=None, must=[FieldCondition(key='country', match=MatchValue(value='Australia'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)], must_not=None)

In [29]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    query_filter=aussie_songs,
    limit=3
)

[ScoredPoint(id=818, version=1, score=0.052790254, payload={'artist': 'Justin Wood', 'country': 'Australia', 'song': 'ability several manage', 'url_song': 'https://adams.info/', 'year': '2003'}, vector=None, shard_key=None),
 ScoredPoint(id=281, version=1, score=-0.1276217, payload={'artist': 'Scott Sweeney', 'country': 'Australia', 'song': 'near discover film', 'url_song': 'https://www.robertson.com/', 'year': '1990'}, vector=None, shard_key=None),
 ScoredPoint(id=292, version=1, score=-0.205366, payload={'artist': 'Lauren White', 'country': 'Australia', 'song': 'of son claim', 'url_song': 'https://www.romero-roberts.com/', 'year': '1976'}, vector=None, shard_key=None)]

In [31]:
## Recommendation API

In [35]:
client.recommend(
    collection_name=my_collection,
    positive=[17, living_la_vida_loca],
    negative=[100, 444],
    query_filter=models.Filter(
    must=[
        models.FieldCondition(
            key="country", match=models.MatchValue(value="Australia")
        )
    ]
),
    limit=5
)

[ScoredPoint(id=818, version=1, score=0.07030617, payload={'artist': 'Justin Wood', 'country': 'Australia', 'song': 'ability several manage', 'url_song': 'https://adams.info/', 'year': '2003'}, vector=None, shard_key=None),
 ScoredPoint(id=281, version=1, score=-0.13908991, payload={'artist': 'Scott Sweeney', 'country': 'Australia', 'song': 'near discover film', 'url_song': 'https://www.robertson.com/', 'year': '1990'}, vector=None, shard_key=None),
 ScoredPoint(id=292, version=1, score=-0.1928386, payload={'artist': 'Lauren White', 'country': 'Australia', 'song': 'of son claim', 'url_song': 'https://www.romero-roberts.com/', 'year': '1976'}, vector=None, shard_key=None),
 ScoredPoint(id=78, version=1, score=-0.2341214, payload={'artist': 'Charles Garrett', 'country': 'Australia', 'song': 'my movement state', 'url_song': 'https://mueller-house.net/', 'year': '2014'}, vector=None, shard_key=None)]